In [ ]:
# |default_exp mongo_episode

In [ ]:
from mongo import print_logs

print_logs(5)

{'_id': ObjectId('6796555bcca048ceac6e4c03'), 'operation': 'update', 'entite': 'episodes', 'desc': '18 déc. 1958 00:00 - Le Masque et les Goncourt : "Saint-Germain ou la Négociation" de Francis Walder, 1958', 'date': datetime.datetime(2025, 1, 26, 16, 31, 39, 156000)}
{'_id': ObjectId('6796555acca048ceac6e4bfa'), 'operation': 'update', 'entite': 'episodes', 'desc': '26 Jan 2025 10:59 - Les nouveaux romans de Leïla Slimani, Pierre Lemaître, Jeanne Rivière, Nathalie Azoulai, Milena Agus', 'date': datetime.datetime(2025, 1, 26, 16, 31, 38, 218000)}
{'_id': ObjectId('67965557cca048ceac6e4bf7'), 'operation': 'delete', 'entite': 'episodes', 'desc': '26 Jan 2025 16:31 - test RSS 1', 'date': datetime.datetime(2025, 1, 26, 16, 31, 35, 288000)}
{'_id': ObjectId('67965557cca048ceac6e4bf4'), 'operation': 'insert', 'entite': 'episodes', 'desc': '26 Jan 2025 16:31 - test RSS 1', 'date': datetime.datetime(2025, 1, 26, 16, 31, 35, 281000)}
{'_id': ObjectId('67965255ca0a9456f50ed099'), 'operation': 'up


# Episode entity

In [ ]:
# |export

import os
from git import Repo

AUDIO_PATH = "audios"


def get_audio_path(audio_path=AUDIO_PATH, year: str = "2024"):
    """
    audio_path: str
        relative path to audio files
    will add year as subdirectory
    return full audio path and create dir if it doesn t exist
    """

    def get_git_root(path):
        git_repo = Repo(path, search_parent_directories=True)
        return git_repo.git.rev_parse("--show-toplevel")

    project_root = get_git_root(os.getcwd())
    full_audio_path = os.path.join(project_root, audio_path, year)

    # create dir if it doesn t exist
    if not os.path.exists(full_audio_path):
        os.makedirs(full_audio_path)

    return full_audio_path

In [ ]:
get_audio_path(AUDIO_PATH)

'/home/guillaume/git/lmelp/audios/2024'

In [ ]:
get_audio_path(AUDIO_PATH, year="")

'/home/guillaume/git/lmelp/audios/'

In [ ]:
# |export

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

import dbus
import time
from functools import wraps


def prevent_sleep(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Connexion au bus D-Bus
        bus = dbus.SessionBus()
        proxy = bus.get_object(
            "org.freedesktop.ScreenSaver", "/org/freedesktop/ScreenSaver"
        )
        interface = dbus.Interface(proxy, "org.freedesktop.ScreenSaver")

        # Prévenir la mise en veille
        cookie = interface.Inhibit("my_script", "Long running process")
        print("Mise en veille désactivée")

        try:
            # Exécuter la fonction décorée
            result = func(*args, **kwargs)
            return result
        finally:
            # Réactiver la mise en veille normale
            interface.UnInhibit(cookie)
            print("Mise en veille normale réactivée")

    return wrapper


@prevent_sleep
def extract_whisper(mp3_filename):

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id = "openai/whisper-large-v3-turbo"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
    )

    dataset = load_dataset(
        "distil-whisper/librispeech_long", "clean", split="validation"
    )
    sample = dataset[0]["audio"]

    result = pipe(
        mp3_filename,
        return_timestamps=True,
    )

    return result["text"]

In [ ]:
# |export

from bson import ObjectId
from mongo import get_collection, get_DB_VARS, mongolog
from datetime import datetime
import requests
from typing import Dict

DATE_FORMAT = "%Y-%m-%dT%H:%M:%S"
LOG_DATE_FORMAT = "%d %b %Y %H:%M"


class Episode:
    def __init__(self, date: str, titre: str, collection_name: str = "episodes"):
        """
        Episode is a class that represents a generic Episode entity in the database.
        :param date: The date for this episode at the format "2024-12-22T09:59:39" parsed by "%Y-%m-%dT%H:%M:%S".
        :param titre: The title of this episode.
        :param collection_name: The name of the collection. default: "episodes".

        if this episode already exists in DB, loads it
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        self.collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )
        self.date = Episode.get_date_from_string(date)
        self.titre = titre

        if self.exists():
            episode = self.collection.find_one({"titre": self.titre, "date": self.date})
            self.description = episode.get("description")
            self.url_telechargement = episode.get("url")
            self.audio_rel_filename = episode.get("audio_rel_filename")
            self.transcription = episode.get("transcription")
            self.type = episode.get("type")
            self.duree = episode.get("duree")
        else:
            self.description = None
            self.url_telechargement = None
            self.audio_rel_filename = None
            self.transcription = None
            self.type = None
            self.duree = -1  # in seconds

    @classmethod
    def from_oid(cls, oid: ObjectId, collection_name: str = "episodes") -> "Episode":
        """
        Create an episode from an oid of a mongo entry.
        :param oid: oid as ObjectId.
        :return: The Eepisode.
        """

        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )

        document = collection.find_one({"_id": oid})

        date_doc_str = cls.get_string_from_date(document.get("date"), DATE_FORMAT)
        inst = cls(date=date_doc_str, titre=document.get("titre"))
        return inst

    @classmethod
    def from_date(cls, date: datetime, collection_name: str = "episodes") -> "Episode":
        """
        Create an episode from a date of a mongo entry.
        :param date: date as datetime.
        :return: The Episode.
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )

        # Convertir la date en début et fin de journée pour la requête
        start_date = datetime(date.year, date.month, date.day)
        end_date = datetime(date.year, date.month, date.day, 23, 59, 59)

        # Rechercher un document dont la date est dans l'intervalle
        document = collection.find_one({"date": {"$gte": start_date, "$lte": end_date}})

        if document:
            date_doc_str = cls.get_string_from_date(document.get("date"), DATE_FORMAT)
            inst = cls(date=date_doc_str, titre=document.get("titre"))
            return inst
        else:
            return None

    def exists(self) -> bool:
        """
        Check if the episode exists in the database.
        :return: True if the episode exists, False otherwise.
        """
        return (
            self.collection.find_one({"titre": self.titre, "date": self.date})
            is not None
        )

    def keep(self) -> int:
        """
        download the audio file if needed
        Keep the episode in the database

        retourne 1 si 1 entree est creee en base
        0 sinon
        """
        message_log = f"{Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} - {self.titre}"
        if not self.exists():
            print(
                f"Episode du {Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} nouveau: Duree: {self.duree}, Type: {self.type}"
            )
            mongolog("insert", self.collection.name, message_log)
            self.download_audio(verbose=True)
            self.collection.insert_one(
                {
                    "titre": self.titre,
                    "date": self.date,
                    "description": self.description,
                    "url": self.url_telechargement,
                    "audio_rel_filename": self.audio_rel_filename,
                    "transcription": self.transcription,
                    "type": self.type,
                    "duree": self.duree,
                }
            )
            return 1
        else:
            print(
                f"Episode du {Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} deja existant"
            )
            mongolog("update", self.collection.name, message_log)
            return 0

    def update_date(self, new_date: datetime):
        """
        Update the date of the episode.
        :param new_date: The new date of the episode, datetime format.
        """
        self.collection.update_one(
            {"_id": self.get_oid()}, {"$set": {"date": new_date}}
        )
        message_log = f"{Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} - {self.titre} -> {Episode.get_string_from_date(new_date, format=LOG_DATE_FORMAT)}"
        self.date = new_date
        mongolog("force_update", self.collection.name, message_log)

    def remove(self):
        """
        Remove the episode from the database.
        """
        message_log = f"{Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} - {self.titre}"
        self.collection.delete_one({"titre": self.titre, "date": self.date})
        mongolog("delete", self.collection.name, message_log)

    def get_oid(self) -> ObjectId:
        """
        Get the object id of the episode.
        :return: The object id of the episode. (bson.ObjectId)
        None if does not exist.
        """
        document = self.collection.find_one({"titre": self.titre, "date": self.date})
        if document:
            return document["_id"]
        else:
            return None

    @staticmethod
    def get_date_from_string(date: str, DATE_FORMAT: str = DATE_FORMAT) -> datetime:
        """
        Get the datetime object from a string.
        :param date: The date string.
        :return: The datetime object.
        """
        return datetime.strptime(date, DATE_FORMAT)

    @staticmethod
    def get_string_from_date(date: datetime, format: str = None) -> str:
        """
        Get the string from a datetime object.
        :param date: The datetime object.
        :param format: The format of the string. default: None and DATE_FORMAT will be used.
        :return: The date string.
        """
        if format is not None:
            return date.strftime(format)
        else:
            return date.strftime(DATE_FORMAT)

    @staticmethod
    def format_duration(seconds: int) -> str:
        """Convert duration in seconds to HH:MM:SS format."""
        if seconds < 0:
            return f"-{Episode.format_duration(seconds*(-1))}"
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = seconds % 60
        return f"{hours:02}:{minutes:02}:{seconds:02}"

    def __str__(self):
        return f"""
        _oid: {self.get_oid()}
        Date: {Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)}
        Titre: {self.titre}
        Description: {self.description}
        URL de téléchargement: {self.url_telechargement}
        Fichier audio: {self.audio_rel_filename}
        Duree: {self.duree} en secondes ({Episode.format_duration(self.duree)})
        Transcription: {self.transcription[:100] if self.transcription else 'No transcription yet available'}...
        """

    def __repr__(self):
        return self.__str__()

    def download_audio(self, verbose=False):
        """
        based on url_telechargement
        will download audio file and store in AUDIO_PATH/year
        """
        if self.url_telechargement is None:
            return
        year = str(self.date.year)
        full_audio_path = get_audio_path(AUDIO_PATH, year)
        full_filename = os.path.join(
            full_audio_path, os.path.basename(self.url_telechargement)
        )
        self.audio_rel_filename = os.path.relpath(
            full_filename, get_audio_path(AUDIO_PATH, year="")
        )
        # Vérification si le fichier existe déjà
        if not os.path.exists(full_filename):
            if verbose:
                print(
                    f"Téléchargement de {self.url_telechargement} vers {full_filename}"
                )
            response = requests.get(self.url_telechargement)
            with open(full_filename, "wb") as file:
                file.write(response.content)
        else:
            if verbose:
                print(f"Le fichier {full_filename} existe déjà. Ignoré.")

    def set_transcription(self, verbose=False, keep_cache=True):
        """
        based on audio file, use whisper model to get transcription
        if transcription already exists, do nothing
        if cache transcription (meaning a txt file aside audio file, same stem name), read it and store in DB
        if audio file does not exist, do nothing
        if keep_cache, save transcription in a txt file aside audio file, same stem name
        save transcription in DB
        """
        if self.transcription is not None:
            if verbose:
                print("Transcription existe deja")
            return
        mp3_fullfilename = get_audio_path(AUDIO_PATH, year="") + self.audio_rel_filename
        cache_transcription_filename = f"{os.path.splitext(mp3_fullfilename)[0]}.txt"
        # check if cache_transcription_file exists
        if os.path.exists(cache_transcription_filename):
            if verbose:
                print(f"Transcription cachee trouvee: {cache_transcription_filename}")
            with open(cache_transcription_filename, "r") as file:
                self.transcription = file.read()
            self.collection.update_one(
                {"_id": self.get_oid()},
                {"$set": {"transcription": self.transcription}},
            )
            return

        self.transcription = extract_whisper(mp3_fullfilename)
        if keep_cache:
            # Écrire la transcription dans un fichier texte
            with open(cache_transcription_filename, "w") as f:
                f.write(self.transcription)
        self.collection.update_one(
            {"_id": self.get_oid()}, {"$set": {"transcription": self.transcription}}
        )

    def to_dict(self) -> Dict[str, str]:
        """
        return episode as a dictionnary
        keys are ['date', 'titre', 'description', 'url_telechargement', 'audio_rel_filename', 'transcription', 'type', 'duree']
        """
        return {
            "date": self.date,
            "titre": self.titre,
            "description": self.description,
            "url_telechargement": self.url_telechargement,
            "audio_rel_filename": self.audio_rel_filename,
            "transcription": self.transcription,
            "type": self.type,
            "duree": self.duree,
        }

In [ ]:
episode = Episode(
    date="2025-01-12T08:59:39",
    titre="Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...",
)
episode


        _oid: None
        Date: 12 Jan 2025 08:59
        Titre: Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...
        Description: None
        URL de téléchargement: None
        Fichier audio: None
        Duree: -1 en secondes (-00:00:01)
        Transcription: No transcription yet available...
        

In [ ]:
episode.update_date(datetime(2025, 1, 12, 8, 59, 39))
episode


        _oid: 678ad2ff9010ec6dc606dce9
        Date: 12 janv. 2025 08:59
        Titre: Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...
        Description: durée : 00:48:41 - Le Masque et la Plume - par : Rebecca Manzoni - Un passé nazi qui refait surface ; une cité magique abrite un amour perdu ; un cinéaste entre un tournage en Afrique et la chute d'un homme nu ; un danseur offre à son fils adoptif la quête d'un héritage familial mouvementé ; un jeune père se confronte à son propre passé douloureux. - invités : Raphaelle Leyris, Hubert ARTUS, Jean-Marc Proust, Elisabeth Philippe - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Hubert Artus : Journaliste et chroniqueur littéraire, Jean-Marc Proust : Auteur et critique (Slate), Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault
        URL de téléchargement: https://rf.proxycast.org/ad97aa2e-ebfc-4d00-8739-4ca72192e726/14007-12.01.2025-ITEMA_2399

In [ ]:
episode.to_dict()

{'date': datetime.datetime(2025, 1, 12, 8, 59, 39),
 'titre': 'Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...',
 'description': None,
 'url_telechargement': None,
 'audio_rel_filename': None,
 'transcription': None,
 'type': None,
 'duree': -1}

In [ ]:
episode = Episode.from_oid(ObjectId("678586de9ff0dfcda11eacf8"))
episode


        _oid: 678586de9ff0dfcda11eacf8
        Date: 22 Dec 2024 09:59
        Titre: Les nouvelles pages de Marc Dugain, Emmanuelle Lambert, Emil Ferris, Fabrice Caro et Mathieu Palain
        Description: durée : 00:47:52 - Le Masque et la Plume - par : Rebecca Manzoni - Il est encore temps d'ajouter quelques livres sous le sapin - invités : Raphaelle Leyris, Jean-Marc Proust, Patricia Martin, Laurent CHALUMEAU - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Jean-Marc Proust : Auteur et critique (Slate), Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier - réalisé par : Guillaume Girault
        URL de téléchargement: https://rf.proxycast.org/7e653bf4-87a5-42f4-864b-9208e206a295/14007-22.12.2024-ITEMA_23973143-2024F4007S0357-22.mp3
        Fichier audio: 2024/14007-22.12.2024-ITEMA_23973143-2024F4007S0357-22.mp3
        Duree: 2872 en secondes (00:47:52)
       

In [ ]:
episode = Episode.from_date(datetime(2025, 1, 12))
episode


        _oid: 678ad2ff9010ec6dc606dce9
        Date: 12 Jan 2025 10:59
        Titre: Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...
        Description: durée : 00:48:41 - Le Masque et la Plume - par : Rebecca Manzoni - Un passé nazi qui refait surface ; une cité magique abrite un amour perdu ; un cinéaste entre un tournage en Afrique et la chute d'un homme nu ; un danseur offre à son fils adoptif la quête d'un héritage familial mouvementé ; un jeune père se confronte à son propre passé douloureux. - invités : Raphaelle Leyris, Hubert ARTUS, Jean-Marc Proust, Elisabeth Philippe - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Hubert Artus : Journaliste et chroniqueur littéraire, Jean-Marc Proust : Auteur et critique (Slate), Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault
        URL de téléchargement: https://rf.proxycast.org/ad97aa2e-ebfc-4d00-8739-4ca72192e726/14007-12.01.2025-ITEMA_239932

# RSS_episode

In [ ]:
# |export

from feedparser.util import FeedParserDict
from transformers import pipeline
import locale

RSS_DUREE_MINI_MINUTES = 15
RSS_DATE_FORMAT = "%a, %d %b %Y %H:%M:%S %z"  # "Sun, 29 Dec 2024 10:59:39 +0100" 'Sun, 26 Jan 2025 10:59:39 +0100'


class RSS_episode(Episode):
    def __init__(self, date: str, titre: str):
        """
        RSS_episode is a class that represents an RSS episode in the database episodes.
        :param date: The date for this episode at the format "2024-12-22T09:59:39" parsed by "%Y-%m-%dT%H:%M:%S".
        :param titre: The title of this episode.
        """
        super().__init__(date, titre)

    @classmethod
    def from_feed_entry(cls, feed_entry: FeedParserDict) -> "RSS_episode":
        """
        Create an RSS episode from a feed entry.
        :param feed_entry: The feed entry.
        :return: The RSS episode.
        """
        locale.setlocale(locale.LC_TIME, "en_US.UTF-8")

        date_rss = datetime.strptime(feed_entry.published, RSS_DATE_FORMAT)
        date_rss_str = cls.get_string_from_date(date_rss, DATE_FORMAT)
        inst = cls(
            date=date_rss_str,
            titre=feed_entry.title,
        )
        inst.description = feed_entry.summary

        for link in feed_entry.links:
            if link.type == "audio/mpeg":
                inst.url_telechargement = link.href
                break

        # self.audio_rel_filename = None
        # self.transcription = None
        inst.type = cls.set_titre(inst.titre + " " + inst.description)
        inst.duree = cls.get_duree_in_seconds(feed_entry.itunes_duration)  # in seconds

        return inst

    @staticmethod
    def get_duree_in_seconds(duree: str) -> int:
        """
        Get the duration in seconds from a string.
        :param duree: The duration string at the format "HH:MM:SS" or "HH:MM".
        :return: The duration in seconds.
        """
        duree = duree.split(":")
        if len(duree) == 3:
            return int(duree[0]) * 3600 + int(duree[1]) * 60 + int(duree[2])
        elif len(duree) == 2:
            return int(duree[0]) * 60 + int(duree[1])
        else:
            return int(duree[0])

    def keep(self) -> int:
        """
        Keep the episode in the database.
        only if duration > RSS_DUREE_MINI_MINUTES * 60
        only if type == livres

        retourne 1 si 1 entree est creee en base
        0 sinon
        """
        if (self.duree > RSS_DUREE_MINI_MINUTES * 60) & (self.type == "livres"):
            return super().keep()
        else:
            print(
                f"Episode du {Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} ignored: Duree: {self.duree}, Type: {self.type}"
            )
            return 0

    @staticmethod
    def set_titre(description: str) -> str:
        """
        use bart meta model from huggingface to classify episodes from
        ["livres", "films", "pièces de théâtre"]
        """
        # Charger le pipeline de classification de texte
        classifier = pipeline(
            "zero-shot-classification", model="facebook/bart-large-mnli"
        )
        # Labels possibles
        labels = ["livres", "films", "pièces de théâtre"]

        result = classifier(description, labels)
        return result["labels"][0]

In [ ]:
from datetime import datetime
import pytz

now = datetime.now(tz=pytz.timezone("Europe/Paris"))

rss1 = RSS_episode(RSS_episode.get_string_from_date(now), "test RSS 1")
print(f"Est-ce que l episode existe ? {rss1.exists()}")

rss1.duree = 4000
rss1.type = "livres"
rss1.keep()
print(f"et maintenant, st-ce que rss1 existe ? {rss1.exists()}")
print(f"et voici l'id de rss1 : {rss1.get_oid()}")

rss1.remove()
print(f"après nettoyage, est-ce que rss1 existe ? {rss1.exists()}")
print(f"et son oid : {rss1.get_oid()}")

Est-ce que l episode existe ? False
Episode du 29 Jan 2025 18:26 nouveau: Duree: 4000, Type: livres
et maintenant, st-ce que rss1 existe ? True
et voici l'id de rss1 : 679a64dc2ab02682c10e62e0
après nettoyage, est-ce que rss1 existe ? False
et son oid : None


In [ ]:
import feedparser
from rss import get_RSS_URL

parsed_flow = feedparser.parse(get_RSS_URL())

rss2 = RSS_episode.from_feed_entry(parsed_flow.entries[0])
rss2

Device set to use cpu



        _oid: 679649e267b093aaae847524
        Date: 26 Jan 2025 10:59
        Titre: Les nouveaux romans de Leïla Slimani, Pierre Lemaître, Jeanne Rivière, Nathalie Azoulai, Milena Agus
        Description: durée : 00:47:56 - Le Masque et la Plume - par : Rebecca Manzoni - Une saga familiale à travers trois générations de femmes, entre le Maroc et la France ; une histoire d'amour et une réflexion sur la judéité ; un roman filial et d'espionnage dans la Guerre Froide ; amitié, désir, musique punk sans les années 90 ; littérature et amour en Sardaigne. - invités : Arnaud Viviant, Laurent CHALUMEAU, Patricia Martin, Elisabeth Philippe - Arnaud Viviant : Critique littéraire (Revue Regards), Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier, Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault
        URL de téléchargement: https://rf.proxycast.org/f421

In [ ]:
rss2.keep()
rss2.exists()

Episode du 26 Jan 2025 10:59 deja existant


True

In [ ]:
# rss2.remove()

In [ ]:
print_logs(5)

{'_id': ObjectId('679a64e02ab02682c10e62e5'), 'operation': 'update', 'entite': 'episodes', 'desc': '26 Jan 2025 10:59 - Les nouveaux romans de Leïla Slimani, Pierre Lemaître, Jeanne Rivière, Nathalie Azoulai, Milena Agus', 'date': datetime.datetime(2025, 1, 29, 18, 26, 56, 53000)}
{'_id': ObjectId('679a64dc2ab02682c10e62e2'), 'operation': 'delete', 'entite': 'episodes', 'desc': '29 Jan 2025 18:26 - test RSS 1', 'date': datetime.datetime(2025, 1, 29, 18, 26, 52, 531000)}
{'_id': ObjectId('679a64dc2ab02682c10e62df'), 'operation': 'insert', 'entite': 'episodes', 'desc': '29 Jan 2025 18:26 - test RSS 1', 'date': datetime.datetime(2025, 1, 29, 18, 26, 52, 525000)}
{'_id': ObjectId('6796555bcca048ceac6e4c03'), 'operation': 'update', 'entite': 'episodes', 'desc': '18 déc. 1958 00:00 - Le Masque et les Goncourt : "Saint-Germain ou la Négociation" de Francis Walder, 1958', 'date': datetime.datetime(2025, 1, 26, 16, 31, 39, 156000)}
{'_id': ObjectId('6796555acca048ceac6e4bfa'), 'operation': 'upd

In [ ]:
episode3 = Episode(
    date="2024-11-10T09:59:39",
    titre="La foire du livre de Brive : les romans de Daniel Pennac, Colson Whitehead, Olivier Norek, Miguel Bonnefoy...",
)
episode3


        _oid: 6773e32258fc5717f3516b9f
        Date: 10 Nov 2024 09:59
        Titre: La foire du livre de Brive : les romans de Daniel Pennac, Colson Whitehead, Olivier Norek, Miguel Bonnefoy...
        Description: durée : 00:46:03 - Le Masque et la Plume - par : Rebecca Manzoni - En direct de la 42ᵉ édition de la Foire du livre de Brive-la-Gaillarde, nos critiques vous disent ce qu'ils ont pensé de "Mon assassin" de Daniel Pennac, "Le rêve du jaguar" de Miguel Bonnefoy, "Les guerriers de l’hiver" d’Olivier Norek, "La règle du crime" de Colson Whitehead...
 - invités : Arnaud Viviant, Elisabeth Philippe, Jean-Marc Proust, Patricia Martin - Arnaud Viviant : Critique littéraire (Revue Regards), Elisabeth Philippe : Critique littéraire (L'Obs), Jean-Marc Proust : Auteur et critique (Slate), Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter - réalisé par : Guillaume Girault
        URL de téléchargement: https://rf.proxycast.org/f5779476-9d52-4bfb-a839-

In [ ]:
episode4 = Episode.from_date(Episode.get_date_from_string("10/11/2024", "%d/%m/%Y"))
episode4


        _oid: 6773e32258fc5717f3516b9f
        Date: 10 Nov 2024 09:59
        Titre: La foire du livre de Brive : les romans de Daniel Pennac, Colson Whitehead, Olivier Norek, Miguel Bonnefoy...
        Description: durée : 00:46:03 - Le Masque et la Plume - par : Rebecca Manzoni - En direct de la 42ᵉ édition de la Foire du livre de Brive-la-Gaillarde, nos critiques vous disent ce qu'ils ont pensé de "Mon assassin" de Daniel Pennac, "Le rêve du jaguar" de Miguel Bonnefoy, "Les guerriers de l’hiver" d’Olivier Norek, "La règle du crime" de Colson Whitehead...
 - invités : Arnaud Viviant, Elisabeth Philippe, Jean-Marc Proust, Patricia Martin - Arnaud Viviant : Critique littéraire (Revue Regards), Elisabeth Philippe : Critique littéraire (L'Obs), Jean-Marc Proust : Auteur et critique (Slate), Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter - réalisé par : Guillaume Girault
        URL de téléchargement: https://rf.proxycast.org/f5779476-9d52-4bfb-a839-

In [ ]:
episode4.transcription = None
episode4.set_transcription(verbose=True)

Transcription cachee trouvee: /home/guillaume/git/lmelp/audios/2024/14007-10.11.2024-ITEMA_23920569-2024F4007S0315-22.txt


In [ ]:
episode4.transcription[:100]

' France Inter Le masque et la plume Bienvenue à Brive la Gaillarde, où se tient la 42e foire du livr'

In [ ]:
episode_tres_court = Episode.from_date(
    Episode.get_date_from_string("27/09/1992", "%d/%m/%Y")
)
episode_tres_court.transcription = None

In [ ]:
episode_tres_court.audio_rel_filename

mp3_fullfilename = (
    get_audio_path(AUDIO_PATH, year="") + episode_tres_court.audio_rel_filename
)
cache_transcription_filename = f"{os.path.splitext(mp3_fullfilename)[0]}.txt"
cache_transcription_filename
# decommenter pour supprimer le fichier cache ce qui lancera le traitement whisper
# !rm {cache_transcription_filename}

'/home/guillaume/git/lmelp/audios/1992/14007-27.09.1992-ITEMA_23787897-2024F4007E0094-27.txt'

In [ ]:
episode_tres_court.set_transcription(verbose=True)

Transcription cachee trouvee: /home/guillaume/git/lmelp/audios/1992/14007-27.09.1992-ITEMA_23787897-2024F4007E0094-27.txt


# WEB_episode

In [ ]:
# |export

import requests
from bs4 import BeautifulSoup
import json
import locale


WEB_DATE_FORMAT = "%d %b %Y"  # '26 août 2024', '20 oct. 2024', '22 sept. 2024', '8 sept. 2024', '25 août 2024', '4 août 2024', '23 juin 2024', '19 mai 2024', '5 mai 2024',


class WEB_episode(Episode):
    def __init__(self, date: str, titre: str):
        """
        WEB_episode is a class that represents an historical episode (legacy, not available anymore as RSS) in the database episodes.
        :param date: The date for this episode at the format "2024-12-22T09:59:39" parsed by "%Y-%m-%dT%H:%M:%S".
        :param titre: The title of this episode.
        """
        super().__init__(date, titre)

    @staticmethod
    def parse_web_date(web_date: str, web_date_format=WEB_DATE_FORMAT) -> datetime:
        """Convertit une date en français dans la page de masque sous forme de chaîne de caractères en un objet datetime.
        la page du masque utilise des abreviations non standards pour fev et juil
        """

        locale.setlocale(locale.LC_TIME, "fr_FR.UTF-8")

        def corrige_date(date_str):
            # Dictionnaire de remplacement pour corriger les abréviations des mois
            month_replacements = {
                "fév.": "févr.",
                "juill.": "juil.",
            }
            for fr_month, fr_month_norm in month_replacements.items():
                date_str = date_str.replace(fr_month, fr_month_norm)
            return date_str

        # Convertir la date normalisée en objet datetime
        try:
            dt = datetime.strptime(corrige_date(web_date), web_date_format)
            return dt
        except ValueError as e:
            print(f"Erreur de conversion pour la date '{web_date}': {e}")
            return None

    @staticmethod
    def get_audio_url(url):
        """
        Prend l'url d'un épisode du masque en entrée et retourne l'URL vers le fichier audio .m4a ou .mp3.
        """

        try:
            # Faire une requête HTTP pour obtenir le contenu de la page
            response = requests.get(url)
            response.raise_for_status()  # Vérifier que la requête a réussi
        except requests.RequestException as e:
            print(f"Erreur lors de la requête HTTP: {e}")
            return None

        # Analyser le contenu HTML avec BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Rechercher la balise <script> contenant l'objet JSON
        script_tag = soup.find("script", string=lambda t: t and "contentUrl" in t)

        if script_tag:
            try:
                # Extraire le contenu JSON de la balise <script>
                json_text = script_tag.string
                json_data = json.loads(json_text)

                # Extraire l'URL du fichier audio
                audio_url = None
                for item in json_data.get("@graph", []):
                    if item.get("@type") == "RadioEpisode":
                        main_entity = item.get("mainEntity", {})
                        audio_url = main_entity.get("contentUrl")
                        break

                return audio_url
            except (json.JSONDecodeError, KeyError, TypeError) as e:
                print(f"Erreur lors de l'analyse du JSON: {e}")
                return None

        print("Balise <script> contenant 'contentUrl' non trouvée")
        return None

    @classmethod
    def from_webpage_entry(cls, dict_web_episode: dict) -> "WEB_episode":
        """
        Create a WEB episode from a dict web episode entry.
        :param dict_web_episode: The web episode entry with these keys: ['title', 'url', 'description', 'date', 'duration']
        :return: The WEB episode.
        """

        date_web = cls.parse_web_date(dict_web_episode["date"])
        date_web_str = cls.get_string_from_date(date_web, DATE_FORMAT)
        inst = cls(
            date=date_web_str,
            titre=dict_web_episode["title"],
        )
        inst.description = dict_web_episode["description"]
        inst.type = "livres"
        inst.url_telechargement = cls.get_audio_url(dict_web_episode["url"])

        # self.audio_rel_filename = None
        # self.transcription = None
        inst.duree = cls.get_duree_in_seconds(
            dict_web_episode["duration"]
        )  # in seconds

        return inst

    @staticmethod
    def get_duree_in_seconds(duree: str) -> int:
        """
        Get the duration in seconds from a string.
        :param duree: The duration string at the format "MM min".
        :return: The duration in seconds.
        """
        duree = duree.split(" ")
        if len(duree) == 2:
            return int(duree[0]) * 60

In [ ]:
from web import WebPage

legacy_episodes = WebPage()
legacy_episodes[-1]

{'title': 'Le Masque et les Goncourt : "Saint-Germain ou la Négociation" de Francis Walder, 1958',
 'url': 'https://www.radiofrance.fr/franceinter/podcasts/le-masque-et-la-plume/francis-walder-saint-germain-ou-la-negociation-2878243',
 'description': 'En\n 1958, les jurés du Prix Goncourt récompensent un écrivain belge, \nFrancis Walder, pour son roman historique "Saint-Germain et la \nnégociation". Les critiques du "Masque et la plume" ont du mal à cacher \nleur déception.',
 'date': '18 déc. 1958',
 'duration': '6 min'}

In [ ]:
vieil_episode = WEB_episode.from_webpage_entry(legacy_episodes[-1])

In [ ]:
vieil_episode


        _oid: 678ccc978ae61760f3ab13b0
        Date: 18 déc. 1958 00:00
        Titre: Le Masque et les Goncourt : "Saint-Germain ou la Négociation" de Francis Walder, 1958
        Description: En
 1958, les jurés du Prix Goncourt récompensent un écrivain belge, 
Francis Walder, pour son roman historique "Saint-Germain et la 
négociation". Les critiques du "Masque et la plume" ont du mal à cacher 
leur déception.
        URL de téléchargement: https://media.radiofrance-podcast.net/podcast09/14007-18.12.1958-ITEMA_23787915-2024F4007E0108-27.m4a
        Fichier audio: 1958/14007-18.12.1958-ITEMA_23787915-2024F4007E0108-27.m4a
        Duree: 360 en secondes (00:06:00)
        Transcription:  France Inter il est évident que nous l'avons profondément mûri et que nous avons lu avec beaucoup d...
        

In [ ]:
vieil_episode.keep()

Episode du 18 déc. 1958 00:00 deja existant


0

# Episodes entity

In [ ]:
# |export

from typing import List, Dict, Any
import pymongo


class Episodes:
    """
    This is a class that will allow search on episodes to manage quality of data

    For example get new transcriptions.
    """

    def __init__(self, collection_name: str = "episodes"):
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        self.collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )
        self.episodes = self._load_episodes()

    def _load_episodes(self) -> List[Dict[str, Any]]:
        """
        Load all episodes from the database and return them as a list of dictionaries.
        """
        return self.get_entries()

    def get_entries(self, request="") -> List[Episode]:
        """'
        retourne le resultat de la requete sous forme d'une liste d'instance de Episode
        tries par date decroissante (du plus recent au plus vieux)

        par exemple : request={"$or": [{"transcription": ""}, {"transcription": None}]}
        """
        result = self.collection.find(request).sort("date", pymongo.DESCENDING)
        episodes = [Episode.from_oid(entry.get("_id")) for entry in result]
        return episodes

    def get_missing_transcriptions(self) -> List[Episode]:
        """
        retourne les episodes pour lesquels la transcription est manquante
        """
        return self.get_entries(
            {"$or": [{"transcription": ""}, {"transcription": None}]}
        )

    def get_transcriptions(self) -> List[Episode]:
        """
        Retourne toutes les entrées pour lesquelles une transcription existe.
        """
        return self.get_entries(
            {"$and": [{"transcription": {"$ne": None}}, {"transcription": {"$ne": ""}}]}
        )

    def __getitem__(self, index: int) -> Dict[str, Any]:
        return self.episodes[index]

    def __len__(self) -> int:
        return len(self.episodes)

    def __iter__(self):
        return iter(self.episodes)

    def __repr__(self) -> str:
        return f"Episodes({self.episodes})"

    def __str__(self):
        return f"""
        {self.collection.count_documents({})} entries
        {len(self.get_missing_transcriptions())} missing transcriptions
        """

    def __repr__(self):
        return self.__str__()

In [ ]:
episodes = Episodes()
episodes.get_missing_transcriptions()
episodes


        206 entries
        0 missing transcriptions
        

In [ ]:
len(episodes)

206

In [ ]:
episodes[0]


        _oid: 679649e267b093aaae847524
        Date: 26 janv. 2025 10:59
        Titre: Les nouveaux romans de Leïla Slimani, Pierre Lemaître, Jeanne Rivière, Nathalie Azoulai, Milena Agus
        Description: durée : 00:47:56 - Le Masque et la Plume - par : Rebecca Manzoni - Une saga familiale à travers trois générations de femmes, entre le Maroc et la France ; une histoire d'amour et une réflexion sur la judéité ; un roman filial et d'espionnage dans la Guerre Froide ; amitié, désir, musique punk sans les années 90 ; littérature et amour en Sardaigne. - invités : Arnaud Viviant, Laurent CHALUMEAU, Patricia Martin, Elisabeth Philippe - Arnaud Viviant : Critique littéraire (Revue Regards), Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier, Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault
        URL de téléchargement: https://rf.proxycast.org/f4

# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("py mongo helper episodes.ipynb", ".")